In [1]:
from quantum_transformers.data import get_imdb_dataloaders
from quantum_transformers.training import train
from quantum_transformers.transformers import ClassicalTransformer

import torch

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.get_device_name() if device.type == 'cuda' else 'cpu'

'NVIDIA A100-SXM4-40GB'

In [3]:
(imdb_train_dataloader, imdb_valid_dataloader), vocab = get_imdb_dataloaders(batch_size=32)
print(imdb_train_dataloader.dataset[0][1])

I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, eve

In [4]:
model = ClassicalTransformer(num_tokens=len(vocab), num_classes=2, hidden_size=64, num_heads=2, num_transformer_blocks=4, mlp_hidden_size=32)
train(model, imdb_train_dataloader, imdb_valid_dataloader, learning_rate=0.0003, num_epochs=30, device=device)

Epoch 1/30 (47.71s): Loss = 0.5629, Accuracy = 71.50%, AUC = 78.39%
Epoch 2/30 (93.60s): Loss = 0.4653, Accuracy = 78.20%, AUC = 86.94%
Epoch 3/30 (139.93s): Loss = 0.4322, Accuracy = 81.23%, AUC = 89.84%
Epoch 4/30 (186.13s): Loss = 0.3880, Accuracy = 83.15%, AUC = 91.23%
Epoch 5/30 (232.08s): Loss = 0.3628, Accuracy = 84.29%, AUC = 92.15%
Epoch 6/30 (277.90s): Loss = 0.3578, Accuracy = 84.68%, AUC = 92.64%
Epoch 7/30 (324.02s): Loss = 0.3512, Accuracy = 85.31%, AUC = 93.02%
Epoch 8/30 (370.17s): Loss = 0.3543, Accuracy = 85.47%, AUC = 93.28%
Epoch 9/30 (416.20s): Loss = 0.3581, Accuracy = 85.15%, AUC = 93.49%
Epoch 10/30 (461.96s): Loss = 0.3945, Accuracy = 85.35%, AUC = 93.63%
Epoch 11/30 (507.68s): Loss = 0.3934, Accuracy = 85.08%, AUC = 93.78%
Epoch 12/30 (553.41s): Loss = 0.3782, Accuracy = 86.10%, AUC = 93.94%
Epoch 13/30 (598.94s): Loss = 0.3867, Accuracy = 85.94%, AUC = 93.94%
Epoch 14/30 (644.90s): Loss = 0.4055, Accuracy = 85.00%, AUC = 93.88%
Epoch 15/30 (690.48s): Loss = 0

In [5]:
model = ClassicalTransformer(num_tokens=len(vocab), num_classes=2, hidden_size=6, num_heads=2, num_transformer_blocks=4, mlp_hidden_size=3)
train(model, imdb_train_dataloader, imdb_valid_dataloader, learning_rate=0.0003, num_epochs=30, device=device)

Epoch 1/30 (43.31s): Loss = 0.6931, Accuracy = 50.24%, AUC = 56.54%
Epoch 2/30 (87.10s): Loss = 0.6916, Accuracy = 50.56%, AUC = 60.83%
Epoch 3/30 (130.62s): Loss = 0.6855, Accuracy = 61.36%, AUC = 65.55%
Epoch 4/30 (174.65s): Loss = 0.6675, Accuracy = 60.00%, AUC = 70.38%
Epoch 5/30 (217.97s): Loss = 0.6434, Accuracy = 64.84%, AUC = 75.32%
Epoch 6/30 (260.87s): Loss = 0.5539, Accuracy = 72.13%, AUC = 79.69%
Epoch 7/30 (304.09s): Loss = 0.6548, Accuracy = 67.15%, AUC = 81.25%
Epoch 8/30 (347.64s): Loss = 0.6399, Accuracy = 68.66%, AUC = 83.38%
Epoch 9/30 (391.12s): Loss = 0.4963, Accuracy = 76.49%, AUC = 85.04%
Epoch 10/30 (434.28s): Loss = 0.5345, Accuracy = 74.92%, AUC = 85.86%
Epoch 11/30 (477.71s): Loss = 0.4709, Accuracy = 78.38%, AUC = 86.83%
Epoch 12/30 (521.25s): Loss = 0.4908, Accuracy = 77.43%, AUC = 87.57%
Epoch 13/30 (564.34s): Loss = 0.4799, Accuracy = 78.34%, AUC = 88.06%
Epoch 14/30 (607.38s): Loss = 0.4812, Accuracy = 78.60%, AUC = 88.50%
Epoch 15/30 (650.93s): Loss = 0